## Prelab 6

In [2]:
# %matplotlib widget
# Imports and setup
from pint import UnitRegistry
import math
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import scipy

# Use pint
units = UnitRegistry()
units.default_format = "~P"

# Define Tesla
ureg = UnitRegistry()
ureg.setup_matplotlib()
Q_ = ureg.Quantity

In [20]:
# Known values
# L_goal = 20*10**(-6) * units.henry
B_max = 0.3 * units.tesla
D_max = 0.35 # Duty Cycle Max
I_rms_cu = 10 * units.amp / (units.mm)**2 # rms
Vo = 10 * units.volt
Vg = 18 * units.volt
alpha = 0.8
Ro = 5 * units.ohm
Mo = Vo/Vg

# A_cmin = 59.1 * (units.mm)**2
# A_coil = (11.1/2 * units.mm)**2 * math.pi


Lg_inches = np.array([0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01,
                      0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018, 0.019,
                      0.020, 0.021, 0.022, 0.023, 0.024, 0.025, 0.026, 0.027]) # inches

Lg = Lg_inches*25.4*units.mm

wires = {
    0.82 * (units.mm)**2: 18,
    0.52 * (units.mm)**2: 20,
    0.33 * (units.mm)**2: 22,
    0.20 * (units.mm)**2: 24 
} # mm^2: awg

# u_o = 1.2566 * 10**(-6) * units.newton/(units.amp)**2  # free space (we only need this value)
# u_r = 3300 * 10**(-6) * units.newton/(units.amp)**2

In [31]:
# Current calculation and wire sizing

a = (Vo/Vg) * (np.sqrt(alpha) - D_max)/D_max
print(f"Our transformer turn ratio is 1:a, where a is {a}.")
M = Mo/a
R = Ro/(a**2)
V = Vo/a


I_pk = 2/np.sqrt(alpha) * (M+1) * (V/R)
Io_pk = I_pk / a

I_rms = math.sqrt(D_max/3) * I_pk
Io_rms = math.sqrt(D_max/3) * Io_pk

min_wire_area = max([I_rms/I_rms_cu, Io_rms/I_rms_cu])
wire_gauge=0

for area in wires:
    if area > min_wire_area:
        wire_gauge = wires[area]
        
print("Min wire gauge:", wire_gauge)

Our transformer turn ratio is 1:a, where a is 0.864170144444311.
Min wire gauge: 22


In [ ]:
# Calculate flux max and relevant values
phi_max = A_cmin * B_max
print("Phi_max:", phi_max)
min_turns = ((L_goal*I_pk)/phi_max).to_base_units()
print("Min turns:", math.ceil(min_turns), "turns (note:this was run with i_rms rather than i_pk, so it was an undercount)")

W_max = 0.5 * L_goal * I_pk**2